In [ ]:
import getpass
import os

# os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

api_key = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(model="gpt-3.5-turbo", api_key= api_key)

from bs4 import SoupStrainer
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("http://environnement.wallonie.be/de/eso/eau_distribution/",),
    bs_kwargs=dict(
        parse_only=SoupStrainer("p"
            #class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()
print("Loaded documents: ", docs)

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
print("Text splits: ", splits)

vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    # Only include the first 500 characters per document chunk
    return "\n\n".join(doc.page_content[:500] + "..." for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("Les eaux de surface prélevées proviennent d'où?")  

Loaded documents:  [Document(metadata={'source': 'http://environnement.wallonie.be/de/eso/eau_distribution/'}, page_content='\xa0Janvier 2018\r\n                    Cinquième révision\n Eau du robinetL’usage de l’eau à des fins alimentaires ou d’hygiène corporelle nécessite une excellente qualité physico-chimique, chimique et microbiologique.L’eau distribuée par réseaux constitue un des produits alimentaires les plus contrôlés en Région wallonne avec plus de 30.000 contrôles par an. Ces derniers sont réalisés depuis son origine jusqu\'au robinet. L’eau de distribution doit répondre aux exigences de qualité imposées par la législation européenne et wallonne. Ainsi, elle ne doit contenir aucun micro-organisme, aucun parasite ni aucune substance constituant un danger potentiel pour la santé des personnes ; elle doit également être conforme vis-à-vis d’un ensemble de normes de potabilité.Origine D’où provient l’eau qui coule de notre  robinet\xa0? En Wallonie, l\'eau potable provient  majo

C:\Users\MediaMonster\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\langsmith\client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


'Task Decomposition is a technique used to break down complex tasks into smaller and simpler steps. It involves instructing a model to "think step by step" to make tasks more manageable. Task decomposition can be enhanced with techniques like Chain of Thought and Tree of Thoughts to improve model performance on complex tasks.'